## Download HCP diffusion data

In [1]:
import os

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os.path as op
%matplotlib inline

/Users/jyeatman/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


### Get the data from AWS S3

We assume that you have a file '~/.aws/credentials', that includes a section: 

    [hcp]
    AWS_ACCESS_KEY_ID=XXXXXXXXXXXXXXXX
    AWS_SECRET_ACCESS_KEY=XXXXXXXXXXXXXXXX
with the credentials you got [from HCP](https://wiki.humanconnectome.org/display/PublicData/How+To+Connect+to+Connectome+Data+via+AWS)

In [12]:
#subjects = [991267, 992774, 994273]
subjects = [991267]

In [13]:
base_dir = op.join(op.expanduser('~'), 'AFQ_data')
if not os.path.exists(base_dir):
    os.mkdir(base_dir)
data_files = {}
for subject in subjects:
    sub_dir = op.join(base_dir, '%s'%subject)
    if not os.path.exists(sub_dir):
        os.mkdir(sub_dir)
        os.mkdir(os.path.join(sub_dir, 'sess'))
        os.mkdir(os.path.join(sub_dir, 'sess', 'dwi'))
        os.mkdir(os.path.join(sub_dir, 'sess', 'anat'))
    data_files['%s/sess/dwi/dwi.bvals'%subject] = 'HCP/%s/T1w/Diffusion/bvals'%subject
    data_files['%s/sess/dwi/dwi.bvecs'%subject] = 'HCP/%s/T1w/Diffusion/bvecs'%subject
    data_files['%s/sess/dwi/dwi.nii.gz'%subject] = 'HCP/%s/T1w/Diffusion/data.nii.gz'%subject
    data_files['%s/sess/anat/T1w_acpc_dc.nii.gz'%subject] = 'HCP/%s/T1w/T1w_acpc_dc.nii.gz'%subject

In [14]:
data_files

{'991267/sess/anat/T1w_acpc_dc.nii.gz': 'HCP/991267/T1w/T1w_acpc_dc.nii.gz',
 '991267/sess/dwi/dwi.bvals': 'HCP/991267/T1w/Diffusion/bvals',
 '991267/sess/dwi/dwi.bvecs': 'HCP/991267/T1w/Diffusion/bvecs',
 '991267/sess/dwi/dwi.nii.gz': 'HCP/991267/T1w/Diffusion/data.nii.gz'}

In [11]:
import botocore.session
import boto3
boto3.setup_default_session(profile_name='hcp')
s3 = boto3.resource('s3')
bucket = s3.Bucket('hcp-openaccess')

In [12]:
for k in data_files.keys():
    if not op.exists(k):
        bucket.download_file(data_files[k], k)